In [ ]:
# !pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
# !pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]

In [ ]:
# !pip install snowflake-connector-python


In [1]:
from snowflake.snowpark import Session

In [9]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=REGISTRY_DATABASE_NAME,
                         schema=REGISTRY_SCHEMA_NAME, 
                         role=parser['Credentials']['role'])
#     print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [10]:
session = get_session()
# session.close()
session

In [8]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWSIGHT"

In [ ]:
# from snowflake.ml.registry import model_registry
from snowflake.ml.registry import Registry as model_registry

In [ ]:
!pip freeze | grep -i cloud

In [ ]:
model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [ ]:
registry = model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

In [ ]:
# registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

In [ ]:
df = registry.show_models()
df
# type(df)

In [ ]:
model_list = registry.models()
model_list

In [ ]:
m =registry.get_model("my_model")
m

In [ ]:
df_version = m.show_versions()
df_version

In [ ]:
# default_version = m.default
m.default = "V2"

In [ ]:
mv = m.default
mv

In [ ]:
#Load default vesion as python object
# clf = mv.load()

In [ ]:
data_list = [ 25, 185,  88,  78,  65,  81,  77,  75,  81,  63,  37,  26,   9,
        10,  79,  82]
data_array = np.asarray(data_list)

remote_prediction = mv.run(data_array, function_name="predict")
remote_prediction   # assuming test_features is Snowpark DataFrame
remote_prediction.tolist()

In [ ]:
# import cloudpickle

# model_obj = cloudpickle.load(open("/notebooks/notebooks/ml_model", "rb"))
# model_obj


In [ ]:
type(x_train)

In [ ]:
# x_train = cloudpickle.load(open("/notebooks/notebooks/x_train", "rb"))
# x_train
# type(x_train)

In [ ]:
mv = registry.log_model(gbm_default,
                   model_name="my_model",
                   version_name="v2",
                   conda_dependencies=["scikit-learn", "cloudpickle"],
                   comment="My awesome ML model",
#                     python_version="3.9",
#                    metrics={"score": 96},
                   sample_input_data=x_train)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Load the data
data = pd.read_csv("/data/fifa.csv")
data.head(10)

In [ ]:
X = data[['age','height_cm','weight_kg','skill_dribbling','attacking_crossing','attacking_finishing','movement_acceleration','movement_sprint_speed','power_shot_power','mentality_aggression','defending_marking','defending_standing_tackle','goalkeeping_diving','goalkeeping_handling','overall','potential']]
y = data['value_eur']
ylog = np.log(y)

# %matplotlib inline
# import matplotlib.pyplot as plt
# plt.hist(ylog, bins='auto')
# plt.title("ln(value_eur)")
# plt.show()

X_train, X_test, ylog_train, ylog_test, y_train, y_test = train_test_split(X, ylog, y, test_size=0.25, random_state=4)

In [ ]:
gbm_default = GradientBoostingRegressor()
gbm_default.fit(X_train, y_train)

# Create stage in snowflake-model-registry

In [ ]:
query = """
        create or replace stage sample_model_serving_udf_stg
        directory = (enable = true)
        copy_options = (on_error='skip_file')
        """

session.sql(query)



In [12]:
# To see the stages
session.sql("show stages").show()

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"created_on"                      |"name"     |"database_name"           |"schema_name"  |"url"  |"has_credentials"  |"has_encryption_key"  |"owner"  |"comment"  |"region"  |"type"    |"cloud"  |"notification_channel"  |"storage_integration"  |"endpoint"  |"owner_role_type"  |"directory_enabled"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2024-05-23 05:38:13.276000-07:00  |ML_MODELS  |MODEL_REGISTRY_SNOWSIGHT  |SNOWSIGHT      |      

# Upload model file on ML_MODELS stage 

In [11]:
#create the stage for storing the ML models
# session.sql("USE MODEL_REGISTRY_SNOWSIGHT;")
session.sql('CREATE OR REPLACE STAGE ML_MODELS').show()

----------------------------------------------
|"status"                                    |
----------------------------------------------
|Stage area ML_MODELS successfully created.  |
----------------------------------------------



In [13]:
session.file.put(
    "/notebooks/notebooks/ml_model", "@ML_MODELS", auto_compress=False, overwrite=True
)

[PutResult(source='ml_model', target='ml_model', source_size=170156, target_size=170160, source_compression='NONE', target_compression='NONE', status='UPLOADED', message='')]

In [14]:
session.clear_imports()
session.clear_packages()

#Register above uploded model as import of UDF
session.add_import("@ML_MODELS/ml_model")

#map packege dependancies
session.add_packages("scikit-learn")

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn'. Your UDF might not work when the package version is different between the server and your local environment.


In [ ]:
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
import snowflake.snowpark.functions as F


def read_file(filename):
    import cloudpickle
    import sys
    import os
    
    #where all imports located at
    import_dir = sys._xoptions.get("snowflake_import_directory")

    if import_dir:
        with open(os.path.join(import_dir, filename), 'rb') as file:
            m = cloudpickle.load(file)
            return m

#register UDF
@F.udf(name = 'predict_score', is_permanent = True, replace = True, stage_location = '@ML_MODELS')
def predict_score(ds: list) -> list:
    
    # later we will input train data as JSON object
    # hance, we have to convert JSON object as pandas DF
    df = pd.io.json.json_normalize(ds)[feature_cols]
    pipeline = read_file('ml_model')
    return pipeline.predict(df)[:,1]

# 